In [1]:
#imports
import numpy as np
import pandas as pd
import random
import string

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os
import csv
import datetime
from datetime import timedelta,datetime, date

# Перезапись CSV в Apache Parquet

In [2]:
transactions = pd.read_csv('transactions.csv')

In [3]:
transactions.START_DATE = pd.to_datetime(transactions.START_DATE)
transactions.END_DATE = pd.to_datetime(transactions.END_DATE)
transactions.DATE = pd.to_datetime(transactions.DATE)
transactions.EXP_TYPE = transactions.EXP_TYPE.astype("category")

In [4]:
transactions.TRANS_ID.duplicated().sum()

0

In [5]:
transactions.drop(columns= ["TRANS_ID"], inplace= True)

In [6]:
cl_to_ids = {}
for (i,v) in enumerate(transactions.CUST_ID.unique()):
  cl_to_ids[v] = i

transactions.CUST_ID = [cl_to_ids[x]  for x in transactions.CUST_ID]

In [7]:
print("Всего клиентов: {}".format(len(transactions['CUST_ID'].unique())))
print("Всего записей: {}".format(len(transactions)))

Всего клиентов: 9835
Всего записей: 29002993


In [8]:
clients = pd.read_csv("customers.csv")
clients.head()

,CUST_ID,PROD_TYPE
0,C8UCZ1WBR2,Накопительные счета
1,C8UCZ1WBR2,Вклады
2,CYSAGMKKKU,Накопительные счета
3,CPU546EFTB,Накопительные счета
4,C5M9RANCEW,Кредиты


In [9]:
clients.PROD_TYPE = clients.PROD_TYPE.astype("category")
clients.CUST_ID = [cl_to_ids[x]  for x in clients.CUST_ID]

In [10]:
print("Всего клиентов: {}".format(len(clients['CUST_ID'].unique())))
print("Всего записей: {}".format(len(clients)))

Всего клиентов: 9835
Всего записей: 13329


In [11]:
#transactions.dropna(inplace= True)
#print("Всего клиентов: {}".format(len(transactions['CUST_ID'].unique())))
#print("Всего записей: {}".format(len(transactions)))

In [12]:
clients = clients.loc[clients.CUST_ID.isin(transactions.CUST_ID.unique()), :]
print("Всего клиентов: {}".format(len(clients['CUST_ID'].unique())))
print("Всего записей: {}".format(len(clients)))

Всего клиентов: 9835
Всего записей: 13329


In [13]:
transactions.YEAR = transactions.YEAR.round().fillna(-1).astype("int")
transactions.MONTH = transactions.MONTH.round().fillna(-1).astype("int")
transactions.DAY = transactions.DAY.round().fillna(-1).astype("int")

In [14]:
transactions.head()

,CUST_ID,START_DATE,END_DATE,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
0,0,2015-03-04,2020-06-27,2017-11-26,2017,11,26,Развлечения,261.99
1,0,2017-01-11,2020-06-27,2020-06-04,2020,6,4,Туристическое агенство,6017.57
2,0,2016-12-19,2020-06-27,2017-09-11,2017,9,11,"Oтели, мотели, курорты",853.71
3,0,2017-02-28,2020-06-27,2018-01-12,2018,1,12,"Билеты кино, музеи, театры, шоу",141.11
4,0,2015-04-06,2020-06-27,2018-01-26,2018,1,26,Развлечения,123.94


In [15]:
clients.head()

,CUST_ID,PROD_TYPE
0,0,Накопительные счета
1,0,Вклады
2,1,Накопительные счета
3,2,Накопительные счета
4,3,Кредиты


In [16]:
transactions.EXP_TYPE.cat.categories[0] == 'Отели, мотели, курорты'

False

### латинские буквы в русских названиях!

In [17]:
import re
cats = transactions.EXP_TYPE.cat.categories.tolist()
[re.match("[, а-яА-Я]+", c) is not None   for c in cats]

[False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [18]:
cats[0] = 'Отели, мотели, курорты'
cats = [x.strip()   for x in cats]
transactions.EXP_TYPE.cat.categories = cats
transactions.EXP_TYPE = 
  transactions.EXP_TYPE.cat.reorder_categories(sorted(cats))

/home/icarus/.local/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:2630: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [19]:
cats = clients.PROD_TYPE.cat.categories.tolist()
[re.match("[, а-яА-Я]+", c) is not None   for c in cats]

[True, True, True, True, True, True, True, True, True, True]

In [20]:
clients.PROD_TYPE.cat.categories = [x.strip()   for x in cats]

In [21]:
np.savetxt("exptypes.txt", transactions.EXP_TYPE.cat.categories.to_numpy(), 
           "%s")
np.savetxt("prodtypes.txt", clients.PROD_TYPE.cat.categories.to_numpy(), 
           "%s")

In [22]:
transactions.to_parquet("transactions.parquet")

In [23]:
clients.to_parquet("customers.parquet")